In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from requests.exceptions import HTTPError


In [2]:
#Funkcja do pobierania wszystkich ofert ze stron

def get_offer_links(url):
    
    is_not_empt_page = True
    page_number = 150
    offer_list = []
    while is_not_empt_page: 
        param = {'page' : page_number}
        try:
            dwl_page = requests.get(url = url , params = param)
            #funkcja nie zwraca niczego jeśli poprawnie wykonała zapytanie 
            dwl_page.raise_for_status()
        except HTTPError as httperror:
            is_not_empt_page = False
            print(f'Nie odnaleziono strony {page_number}.  Blad: {httperror}')
        
        ##pobieranie adresów linków do sprawdzenia:
        soup = BeautifulSoup(dwl_page.text , 'lxml')
        body = soup.body
        div = body.find_all('a', {'class': 'teaser__anchor'})
        
        for i in div:
            offer_list.append(i.get('href'))
        page_number += 1
        #print(page_number)
    
    return offer_list
        
        

In [3]:
lista_ofert_gr = get_offer_links('https://gratka.pl/nieruchomosci/mieszkania/poznan/sprzedaz')

Nie odnaleziono strony 152.  Blad: 404 Client Error: Not Found for url: https://gratka.pl/nieruchomosci/mieszkania/poznan/sprzedaz?page=152


In [4]:
def get_offer_attributes(url):
    offer_page = requests.get(url = url)
    soup = BeautifulSoup(offer_page.text, 'lxml')
    body = soup.body
    
    #params loop
    pars_atrr = body.find('ul', {'class': 'parameters__rolled'})
    #getting list of params 
    params_list = [i.get_text() for i in pars_atrr.find_all('span')]
    params_list.remove('tel.')
    #getting list of values
    values_list = [i.get_text() for i in pars_atrr.find_all('b', class_ = 'parameters__value')]

    #stworzyłem słownik do przechowywania parametów pobranych ze strony 
    dict_object = {key : value for key, value in zip(params_list, values_list)}
    #pobieram cene mieszkania
    dict_object['price'] = body.find('span', class_ = 'priceInfo__value').get_text()
    #pobieram tytuł ogłoszenia
    dict_object['title'] = body.find('h1', class_ = 'sticker__title').get_text()
    dict_object['link'] = url
    
    return dict_object

In [9]:
#funkcja do pobieranie danych z ofert 
def download_data(lista):
    dict_offers = {}
    for count, i in enumerate(lista):
        dict_offers[count] = get_offer_attributes(i)
    
    return dict_offers


In [10]:
data_dict = download_data(lista_ofert_gr)

#  count = 0
#     dict_offers = {}
#     for i in lista_ofert_gr[15:20]:
#         dict_offers[count] = get_offer_attributes(lista)
#         count += 1 


In [11]:
data_df = pd.DataFrame(data_dict)
data_df.transpose()

,Forma / wyposażenie łazienki,Forma kuchni,Forma własności,Głośność,Komunikacja,Liczba pięter w budynku,Liczba pokoi,Lokalizacja,Materiał budynku,Miejsce parkingowe,...,"Opłaty (czynsz administracyjny, media)",Piętro,Powierzchnia w m2,Rok budowy,Stan,Typ zabudowy,link,price,tel.,title
0,NaN,NaN,NaN,NaN,NaN,6,2,"\nPoznań, ...",NaN,NaN,...,NaN,1,42 m2,NaN,NaN,kamienica,https://gratka.pl/nieruchomosci/nowe-mieszkani...,\n Zapytaj o cenę\n,NaN,"Nowe mieszkanie Poznań Rynek Wildecki, ul. Ryn..."
1,NaN,NaN,NaN,NaN,NaN,6,2,"\nPoznań, ...",NaN,NaN,...,NaN,1,42 m2,NaN,NaN,kamienica,https://gratka.pl/nieruchomosci/nowe-mieszkani...,\n Zapytaj o cenę\n,NaN,"Nowe mieszkanie Poznań Rynek Wildecki, ul. Ryn..."
2,NaN,w aneksie,NaN,NaN,NaN,5,4,"\nPoznań, ...",NaN,NaN,...,NaN,5,77 m2,NaN,NaN,blok,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"\n 533 717,00\n \n zł\n ...",NaN,"Nowe mieszkanie Poznań Dolna Wilda, ul. Sapers..."
3,NaN,w aneksie,NaN,NaN,NaN,5,2,"\nPoznań, ...",NaN,NaN,...,NaN,3,44 m2,NaN,NaN,blok,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"\n 329 065,00\n \n zł\n ...",NaN,"Nowe mieszkanie Poznań Dolna Wilda, ul. Sapers..."
4,NaN,w aneksie,NaN,NaN,NaN,5,4,"\nPoznań, ...",NaN,NaN,...,NaN,3,77 m2,NaN,NaN,blok,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"\n 518 974,00\n \n zł\n ...",NaN,"Nowe mieszkanie Poznań Dolna Wilda, ul. Sapers..."
5,NaN,NaN,NaN,NaN,NaN,4,4,"\nPoznań, ...",NaN,NaN,...,NaN,4,90 m2,NaN,NaN,blok,https://gratka.pl/nieruchomosci/mieszkanie-poz...,"\n 390 000,00\n \n zł\n ...",NaN,"Mieszkanie Poznań Antoninek, ul. Poznań Nowe M..."
6,NaN,w aneksie,NaN,NaN,NaN,5,4,"\nPoznań, ...",NaN,NaN,...,NaN,5,70 m2,NaN,NaN,blok,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"\n 488 877,00\n \n zł\n ...",NaN,"Nowe mieszkanie Poznań Dolna Wilda, ul. Sapers..."
7,NaN,w aneksie,NaN,NaN,NaN,5,3,"\nPoznań, ...",NaN,NaN,...,NaN,2,57 m2,NaN,NaN,blok,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"\n 409 100,00\n \n zł\n ...",NaN,"Nowe mieszkanie Poznań Dolna Wilda, ul. Sapers..."
8,NaN,w aneksie,NaN,NaN,NaN,5,3,"\nPoznań, ...",NaN,NaN,...,NaN,1,57 m2,NaN,NaN,blok,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"\n 401 288,00\n \n zł\n ...",NaN,"Nowe mieszkanie Poznań Dolna Wilda, ul. Sapers..."
9,NaN,w aneksie,NaN,NaN,NaN,5,3,"\nPoznań, ...",NaN,NaN,...,NaN,1,59 m2,NaN,NaN,blok,https://gratka.pl/nieruchomosci/nowe-mieszkani...,"\n 418 667,00\n \n zł\n ...",NaN,"Nowe mieszkanie Poznań Dolna Wilda, ul. Sapers..."


In [ ]:
##usunąć dwa telefony podczas pobierania
https://gratka.pl/nieruchomosci/mieszkanie-poznan-grunwald-ul-bulgarska-59/ob/11770245

In [4]:
#liczba podstron z ofertami
start_pg_nr = int(body.find('input', {'class': 'pagination__input'}).get('min'))
end_pg_nr = int(body.find('input', {'class': 'pagination__input'}).get('max'))

In [65]:
class DataScrappingOffers:
    
    def __init__(self, url):
        self.url = url
        
        
    def get_offer_links(self):

        is_not_empt_page = True
        page_number = 140
        offer_list = []
        while is_not_empt_page: 
            param = {'page' : page_number}
            try:
                dwl_page = requests.get(url = self.url , params = param)
                #funkcja nie zwraca niczego jeśli poprawnie wykonała zapytanie 
                dwl_page.raise_for_status()
            except HTTPError as httperror:
                is_not_empt_page = False
                print(f'Nie odnaleziono strony {page_number}.  Blad: {httperror}')

            ##pobieranie adresów linków do sprawdzenia:
            soup = BeautifulSoup(dwl_page.text , 'lxml')
            body = soup.body
            div = body.find_all('a', {'class': 'teaser__anchor'})

            for i in div:
                offer_list.append(i.get('href'))
            page_number += 1
            #print(page_number)

        return offer_list
        

In [73]:
class DataScrappingOffersAtrr:
    
    def __init__(self, url):
        self.url = url
        
    @staticmethod 
    def get_offer_attributes(url):
        offer_page = requests.get(url = url)
        soup = BeautifulSoup(offer_page.text, 'lxml')
        body = soup.body

        #params loop
        pars_atrr = body.find('ul', {'class': 'parameters__rolled'})
        #getting list of params 
        params_list = [i.get_text() for i in pars_atrr.find_all('span')]
        params_list.remove('tel.')
        #getting list of values
        values_list = [i.get_text() for i in pars_atrr.find_all('b', class_ = 'parameters__value')]

        #stworzyłem słownik do przechowywania parametów pobranych ze strony 
        dict_object = {key : value for key, value in zip(params_list, values_list)}
        #pobieram cene mieszkania
        dict_object['price'] = body.find('span', class_ = 'priceInfo__value').get_text()
        #pobieram tytuł ogłoszenia
        dict_object['title'] = body.find('h1', class_ = 'sticker__title').get_text()
        dict_object['link'] = url

        return dict_object



In [67]:
list_of_offers = DataScrappingOffers('https://gratka.pl/nieruchomosci/mieszkania/poznan/sprzedaz')

In [68]:
list_of_offers = list_of_offers.get_offer_links()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gratka.pl:443
DEBUG:urllib3.connectionpool:https://gratka.pl:443 "GET /nieruchomosci/mieszkania/poznan/sprzedaz?page=140 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gratka.pl:443
DEBUG:urllib3.connectionpool:https://gratka.pl:443 "GET /nieruchomosci/mieszkania/poznan/sprzedaz?page=141 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gratka.pl:443
DEBUG:urllib3.connectionpool:https://gratka.pl:443 "GET /nieruchomosci/mieszkania/poznan/sprzedaz?page=142 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gratka.pl:443
DEBUG:urllib3.connectionpool:https://gratka.pl:443 "GET /nieruchomosci/mieszkania/poznan/sprzedaz?page=143 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gratka.pl:443
DEBUG:urllib3.connectionpool:https://gratka.pl:443 "GET /nieruchomosci/mieszkania/poznan/sprzedaz?page=144

In [71]:
#funkcja do pobieranie danych z ofert 
def download_data(lista):
    dict_offers = {}
    for count, i in enumerate(lista):
        dict_offers[count] = DataScrappingOffersAtrr.get_offer_attributes(i)
    
    return dict_offers

In [74]:
download_data(list_of_offers)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gratka.pl:443
DEBUG:urllib3.connectionpool:https://gratka.pl:443 "GET /nieruchomosci/mieszkanie-poznan-centrum-ul-kutrzeby/ob/7812973 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gratka.pl:443
DEBUG:urllib3.connectionpool:https://gratka.pl:443 "GET /nieruchomosci/nowe-mieszkanie-poznan-jezyce-ul-meissnera-2/ob/7087059 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gratka.pl:443
DEBUG:urllib3.connectionpool:https://gratka.pl:443 "GET /nieruchomosci/nowe-mieszkanie-poznan-jezyce-ul-meissnera-2/ob/7087129 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gratka.pl:443
DEBUG:urllib3.connectionpool:https://gratka.pl:443 "GET /nieruchomosci/nowe-mieszkanie-poznan-jezyce-ul-meissnera-2/ob/7090937 HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gratka.pl:443
DEBUG:urllib3.connectionpool:http

KeyboardInterrupt: 